In [6]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import json
import re
import lxml.html as lh
import datetime as dt

ModuleNotFoundError: No module named 'lxml'

### Global confirmed corona cases


In [63]:
df = pd.read_csv('COVID19/time_series_covid19_confirmed_global.csv')
df

FileNotFoundError: [Errno 2] File time_series_covid19_confirmed_global.csv does not exist: 'time_series_covid19_confirmed_global.csv'

In [ ]:
df['Country/Region'].unique()

In [ ]:
data_EU_US = df[df['Country/Region'].isin(['Andorra', 'Albania', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
                                          'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 
                                          'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Liechtenstein',
                                          'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands',
                                          'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino',
                                          'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
                                          'United Kingdom', 'US'])]
data_EU_US

In [ ]:
df_countries_only = data_EU_US[data_EU_US['Province/State'].isna()]
df_countries_only 

In [ ]:
data_clean = df_countries_only.drop(['Province/State'], axis=1)
data_clean

In [ ]:
df_good = data_clean.reset_index()
del df_good['index']
df_good

In [ ]:
df_good.describe()

### Global Deaths corona


In [2]:
df_d = pd.read_csv('time_series_covid19_deaths_global.csv')
df_d

NameError: name 'pd' is not defined

In [8]:
data_EU_US_d = df_d[df_d['Country/Region'].isin(['Andorra', 'Albania', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
                                          'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 
                                          'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Liechtenstein',
                                          'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands',
                                          'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino',
                                          'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine',
                                          'United Kingdom', 'US', 'Venezuela'])]
data_EU_US_d

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,23,23,23,23,24,25,26,26,26,26
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,26,26,29,29,31,33,33,35,35,36
16,NaN,Austria,47.5162,14.5501,0,0,0,0,0,0,...,319,337,350,368,384,393,410,431,443,452
22,NaN,Belarus,53.7098,27.9534,0,0,0,0,0,0,...,19,23,26,29,33,36,40,42,45,47
23,NaN,Belgium,50.8333,4.0000,0,0,0,0,0,0,...,3019,3346,3600,3903,4157,4440,4857,5163,5453,5683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,British Virgin Islands,United Kingdom,18.4207,-64.6400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
251,Turks and Caicos Islands,United Kingdom,21.6940,-71.7979,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
256,"Bonaire, Sint Eustatius and Saba",Netherlands,12.1784,-68.2385,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
258,Falkland Islands (Malvinas),United Kingdom,-51.7963,-59.5236,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_countries_only_d = data_EU_US_d[data_EU_US_d['Province/State'].isna()]
df_countries_only_d 

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,23,23,23,23,24,25,26,26,26,26
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,26,26,29,29,31,33,33,35,35,36
16,NaN,Austria,47.5162,14.5501,0,0,0,0,0,0,...,319,337,350,368,384,393,410,431,443,452
22,NaN,Belarus,53.7098,27.9534,0,0,0,0,0,0,...,19,23,26,29,33,36,40,42,45,47
23,NaN,Belgium,50.8333,4.0000,0,0,0,0,0,0,...,3019,3346,3600,3903,4157,4440,4857,5163,5453,5683
27,NaN,Bosnia and Herzegovina,43.9159,17.6791,0,0,0,0,0,0,...,36,37,39,39,40,41,43,46,47,48
30,NaN,Bulgaria,42.7339,25.4858,0,0,0,0,0,0,...,25,28,29,32,35,36,38,41,41,42
87,NaN,Croatia,45.1000,15.2000,0,0,0,0,0,0,...,21,21,23,25,31,33,35,36,39,47
90,NaN,Cyprus,35.1264,33.4299,0,0,0,0,0,0,...,10,10,11,12,12,12,12,12,12,12
91,NaN,Czechia,49.8175,15.4730,0,0,0,0,0,0,...,119,129,138,143,161,166,169,173,181,186


In [10]:
data_clean_d = df_countries_only_d.drop(['Province/State'], axis=1)
data_clean_d

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20
1,Albania,41.1533,20.1683,0,0,0,0,0,0,0,...,23,23,23,23,24,25,26,26,26,26
3,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,...,26,26,29,29,31,33,33,35,35,36
16,Austria,47.5162,14.5501,0,0,0,0,0,0,0,...,319,337,350,368,384,393,410,431,443,452
22,Belarus,53.7098,27.9534,0,0,0,0,0,0,0,...,19,23,26,29,33,36,40,42,45,47
23,Belgium,50.8333,4.0000,0,0,0,0,0,0,0,...,3019,3346,3600,3903,4157,4440,4857,5163,5453,5683
27,Bosnia and Herzegovina,43.9159,17.6791,0,0,0,0,0,0,0,...,36,37,39,39,40,41,43,46,47,48
30,Bulgaria,42.7339,25.4858,0,0,0,0,0,0,0,...,25,28,29,32,35,36,38,41,41,42
87,Croatia,45.1000,15.2000,0,0,0,0,0,0,0,...,21,21,23,25,31,33,35,36,39,47
90,Cyprus,35.1264,33.4299,0,0,0,0,0,0,0,...,10,10,11,12,12,12,12,12,12,12
91,Czechia,49.8175,15.4730,0,0,0,0,0,0,0,...,119,129,138,143,161,166,169,173,181,186


In [53]:
df_good_d = data_clean_d.reset_index()
del df_good_d['index']
df_good_d
df_good_d = df_good_d.drop(columns=['Lat', 'Long']).set_index("Country/Region")
df_good_d = df_good_d.T.reset_index()
df_good_d = df_good_d.rename(columns={'index':'Date'})
df_good_d.Date = df_good_d.Date.astype('datetime64')
df_good_d

Country/Region,Date,Albania,Andorra,Austria,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,...,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom,US,Venezuela
0,2020-01-22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-01-25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2020-04-15,25,33,393,36,4440,41,36,33,12,...,6,61,18708,1203,1239,1518,108,12868,28325,9
85,2020-04-16,26,33,410,40,4857,43,38,35,12,...,8,61,19315,1333,1281,1643,116,13729,32916,9
86,2020-04-17,26,35,431,42,5163,46,41,36,12,...,9,66,20002,1400,1327,1769,125,14576,36773,9
87,2020-04-18,26,35,443,45,5453,47,41,39,12,...,11,70,20043,1511,1368,1890,133,15464,38664,9


In [65]:
# Creating Eurozone column
df_good_d['Eurozone'] = (df_good_d['Andorra'] + df_good_d['Austria'] + df_good_d['Belgium'] + 
                         df_good_d['Cyprus'] + df_good_d['Estonia'] + df_good_d['Finland'] +
                         df_good_d['France'] + df_good_d['Germany'] + df_good_d['Greece'] + 
                         df_good_d['Ireland'] + df_good_d['Italy'] + df_good_d['Latvia'] + 
                         df_good_d['Liechtenstein'] + df_good_d['Lithuania'] + df_good_d['Luxembourg'] +
                         df_good_d['Malta'] + df_good_d['Monaco'] + df_good_d['Montenegro'] +
                         df_good_d['Netherlands'] + df_good_d['Portugal'] + df_good_d['San Marino'] +
                         df_good_d['Slovakia'] + df_good_d['Slovenia'] + df_good_d['Spain'])

In [66]:
df_good_d.describe()

Country/Region,Albania,Andorra,Austria,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Cyprus,Czechia,...,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom,US,Venezuela,Eurozone
count,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,...,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,5.752809,5.876404,71.977528,4.707865,684.359551,7.719101,6.539326,5.258427,2.730337,26.000000,...,10.393258,3954.539326,188.393258,236.573034,236.449438,16.382022,2015.797753,4455.898876,1.820225,14420.640449
std,9.122002,10.917896,132.141107,11.334984,1439.319641,14.512525,11.752230,10.602881,4.433253,52.433941,...,19.831459,6546.955064,388.795273,415.206216,495.721192,34.355265,4166.166974,9635.301782,3.342433,23890.256565
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,5.000000,0.000000,1.000000,0.000000,0.000000,2.000000,14.000000,0.000000,213.000000
75%,10.000000,3.000000,68.000000,0.000000,353.000000,5.000000,7.000000,5.000000,5.000000,11.000000,...,9.000000,5982.000000,105.000000,264.000000,108.000000,9.000000,1019.000000,2026.000000,2.000000,20055.000000
max,26.000000,36.000000,452.000000,47.000000,5683.000000,48.000000,42.000000,47.000000,12.000000,186.000000,...,74.000000,20453.000000,1540.000000,1393.000000,2017.000000,141.000000,16060.000000,40661.000000,9.000000,80105.000000
